In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import requests
import json
import pandas as pd
import re

In [2]:
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

In [3]:
options = Options()
options.headless = True
DRIVER_PATH = r'/usr/bin/chromedriver'
service = Service(DRIVER_PATH)
driver= webdriver.Chrome(service= service, options=options)

In [4]:
def is_quoraLink(url):
    is_quora_link = re.match(r'https://www.quora.com/', url)
    if(is_quora_link):
        return True
    else: 
        return False
    
# print(is_quoraLink())

In [5]:
def search_google(search_keyword):
    time.sleep(2)
    driver.get("https://google.com/search?q="+search_keyword+ "&start="+str(0))
    title_links = driver.find_elements(By.XPATH, "//div[@class='yuRUbf']/a")
    titles = []
    title_url = []
    for x in title_links:
        if(is_quoraLink(x.get_attribute("href"))):
            time.sleep(5)
            titles.append(x.text.split('\n')[0].strip())
            title_url.append(x.get_attribute("href"))
    
    return titles, title_url

    
# print(search_google('Overseas+education+search+on+quora'))


In [6]:
def search_quora(urls):
    answers = []
    upvotes = []
    for url in urls:
        time.sleep(2)
        req = requests.get(url)
        soup = BeautifulSoup(req.content, 'html.parser')
        resp = json.loads(soup.find('script', type='application/ld+json').text)
        ans = []
        upvote = []
        for x in resp:
            if(x=='mainEntity'):
                for x in resp['mainEntity']['suggestedAnswer']:
                    ans.append(x['text'])
                    upvote.append(x['upvoteCount'])

        answers.append(ans)
        upvotes.append(upvote)
    
    return answers, upvotes

In [12]:
def represent_data(data, key):
    df = pd.DataFrame(data)
    df.to_csv(key+".csv")
    display(df)

In [13]:
KEYWORDS = ["Study abroad",
"International Education", 
"Overseas education", 
"overseas education for Indian students",
"Indian students in Abroad",
"Study in Canada",
"Study in USA",
"Study in UK", 
"Study in Australia", 
"Abroad education", 
"MBA from USA",
"Diploma in Canada",
"PR in canada"]

# KEYWORDS = ["top 5 college to study abroad", 
#             "best country to study abroad", 
#             "top 5 college to Study in Canada",
#            "how to apply for Study in Canada"]

In [ ]:
for key in KEYWORDS:
    time.sleep(5)
    search_keyword = key +" search on quora"
    search_keyword = search_keyword.replace(' ', '+') 
    title, url = search_google(search_keyword)
    ans, upvote = search_quora(url)
    data ={'TITLE':title, 'URL':url, 'ANSWER': ans, 'UPVOTE': upvote}
    represent_data(data, key)